In [2]:
# !pip install yahoo_fin
from yahoo_fin.stock_info import get_data
import pandas as pd

In [3]:
# write a function to loop through the list of tickers and get the data
def get_stock_data(ticker_list, start_date, end_date):
    data = pd.DataFrame()
    bad_tickers = []
    for ticker in ticker_list:
        try: 
            df = get_data(ticker=ticker, start_date=start_date, end_date=end_date, 
                          index_as_date=False, interval='1d')
            df = df[['date', 'ticker', 'adjclose']]
            data = pd.concat([data, df], axis=0)
        except:
            bad_tickers.append(ticker)
            continue
        
    data['date'] = pd.to_datetime(data['date'])
    return data, bad_tickers

# data = get_stock_data(['amzn'], '04/29/2014', '04/28/2024')
# data

In [4]:
transactions = pd.read_csv('../../data/inputs/transactions.csv')
# get all unique tickers from the transactions dataframe
tickers = transactions['ticker'].unique()
# get the stock data for all the tickers
data, bad_tickers = get_stock_data(tickers, '04/29/2014', '04/28/2024')

In [6]:
# save data to stock_prices.csv
data.to_csv('../../data/inputs/stock_prices.csv', index=False)
# save bad_tickers text file separated by comma
with open('../../data/inputs/bad_tickers.txt', 'w') as f:
    for ticker in bad_tickers:
        f.write(ticker + ',')
f.close()